In [2]:
import chess
board = chess.Board("rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1")

In [13]:
for square in board.attacks(chess.B1):
    print(chess.square_name(square))


d2
a3
c3


In [53]:
def evaluate(self, board) -> float:

        piece_values = {
            chess.PAWN: 1,
            chess.KNIGHT: 3,
            chess.BISHOP: 3.15,
            chess.ROOK: 5.1,
            chess.QUEEN: 9,
            chess.KING: 9999 # King is worth "infinite" points
        }

        def get_positional_bonus(piece, square):
            # Simple positional bonuses for pieces
            bonus = 0
            if piece.piece_type == chess.PAWN:
                if piece.color == chess.WHITE:
                    distance_to_promotion = 7 - chess.square_rank(square)
                else:
                    distance_to_promotion = chess.square_rank(square)
                if distance_to_promotion < 6: # Dont give bonus if pawn is on starting square, gives incentive to move pawns
                    bonus += 2 * (3 ** (1 - distance_to_promotion)) # Exponential bonus for pawns closer to promotion - I used 2 when the pawn is 1 square away from promotion because people typically equate 2 pawns on the 7th rank to a rook
            elif piece.piece_type == chess.KNIGHT:
                controlled_squares = board.attacks(square)
                for square in controlled_squares:
                    if board.piece_at(square) is not None:
                        bonus += 0.03
                    bonus += 0.025 * (4 - abs(3 - chess.square_file(square)))  # Bonus for controlling the center
                #bonus += 0.025 * controlling_squares  # Bonus for controlling squares
                #bonus += 0.2 * (4 - abs(3 - chess.file_index(square)))  # Bonus for controlling the center
            elif piece.piece_type == chess.BISHOP:
                controlled_squares = board.attacks(square)
                bonus += 0.02 * len(controlled_squares)  # Bonus for controlling long diagonals
                for attacked_square in controlled_squares:
                    if board.piece_at(attacked_square) is not None and board.is_pinned(not piece.color, attacked_square):
                        bonus += 0.2 # Bonus for pinning pieces
            elif piece.piece_type == chess.ROOK:
                controlled_squares = board.attacks(square)
                bonus += 0.025 * len(controlled_squares)           
                rook_file = chess.square_file(square)
                for pawn_square in board.pieces(chess.PAWN, piece.color):
                    if chess.square_file(pawn_square) == rook_file:
                        bonus -= 0.2 # Penalty for rook being blocked by same coloed pawn
                        break
            elif piece.piece_type == chess.QUEEN:
                controlling_squares = len(board.attacks(square))
                bonus += 0.015 * controlling_squares   # Give queen a smaller bonus since queens have more mobility 
            elif piece.piece_type == chess.KING:
                for attacked_square in board.attacks(square):
                    adjacent_piece = board.piece_at(attacked_square)
                    if adjacent_piece is not None:
                        if adjacent_piece.piece_type == chess.PAWN and adjacent_piece.color == piece.color:
                            bonus += 0.05 # Bonus for king being protected by pawn
                        bonus += 0.02 # Bonus for king being protected by other pieces
            return bonus

        if board.is_checkmate():
            if board.turn:
                return float('-inf')
            else:
                return float('inf')
        elif board.is_stalemate() or board.is_insufficient_material():
            return 0
        else:
            score = 0
            for square in chess.SQUARES:
                piece = board.piece_at(square)
                if piece is not None:
                    value = piece_values[piece.piece_type] + get_positional_bonus(piece, square)
                    if piece.color == chess.WHITE:
                        score += value
                    else:
                        score -= value
                    # print(f"Piece: {piece}, Score: {value}")
            return score

In [58]:
my_game = chess.Board('rnb1kb1r/pp1P1ppp/1qp1P3/3p2B1/4P3/2N3P1/PPP3P1/R2QKBNR')

In [61]:
chess.square_file(chess.B3)

1